# Purpose

Find a way to get data from Plugshare.com since they're not responding to my API access request. The comments and metadata from stations across different networks should be extremely useful in diagnosing electrical and non-electrical customer experience issues.

# Imports

In [1]:

%load_ext autoreload
%autoreload 2

import numpy as np
from rich import print
import pandas as pd

# Electrify America in Springfield, VA mall parking lot
TEST_LOCATION = 252784

from dotenv import load_dotenv
load_dotenv(override=True)

from evlens.logs import setup_logger
logger = setup_logger("Notebook-2.0")
logger.info("TEST!")

2024-07-29_T15_50_01EDT: INFO (Notebook-2.0:L16) - TEST!


In [2]:
pd.options.display.max_columns = 100
pd.options.display.max_rows = 100

# Trying out Folium

I hear it's a very capable high-quality mapping tool, so let's give it a whirl! We'll do everything initially with OpenStreetMap and open source routing to get a feel for things (may eventually switch to Google Routes API).

In [12]:
import folium

WHITEHOUSE_COORDS = (38.8964457,-77.035921) # for funsies
BOSTON_COORDS = (42.3144474,-71.0526845)

start_point_map = folium.Map(location=WHITEHOUSE_COORDS, zoom_start=16, tiles='OpenStreetMap')

# Give it a marker at our start point
folium.Marker(WHITEHOUSE_COORDS, draggable=True).add_to(start_point_map)
start_point_map